In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)

In [3]:
biomart = pd.read_csv("AthBiomart.txt", sep="\t")
biomart.head()

,Gene stable ID,Transcript stable ID,Gene name,Gene description,Affymetrix array Arabidopsis ATH1 121501 ID,GO term accession,GO term name
0,AT3G18710,AT3G18710.1,PUB29,U-box domain-containing protein 29 [Source:Uni...,257748_at,GO:0005488,binding
1,AT3G18710,AT3G18710.1,PUB29,U-box domain-containing protein 29 [Source:Uni...,257748_at,GO:0010200,response to chitin
2,AT3G18710,AT3G18710.1,PUB29,U-box domain-containing protein 29 [Source:Uni...,257748_at,GO:0004842,ubiquitin-protein transferase activity
3,AT3G18710,AT3G18710.1,PUB29,U-box domain-containing protein 29 [Source:Uni...,257748_at,GO:0016874,ligase activity
4,AT3G18710,AT3G18710.1,PUB29,U-box domain-containing protein 29 [Source:Uni...,257748_at,GO:0016567,protein ubiquitination


In [4]:
#MY CONNECTION TO DB
con = sqlite3.connect("td2201_midterm.sqlite")
c = con.cursor()

In [52]:
#number of tables in my db
pd.read_sql("""
            SELECT * FROM sqlite_master
            WHERE type='table';
""", con)

,type,name,tbl_name,rootpage,sql
0,table,DataType,DataType,2,CREATE TABLE DataType(DTID INT PRIMARY KEY NOT...
1,table,Data,Data,4,CREATE TABLE Data(DID INT PRIMARY KEY NOT NULL...
2,table,biomart,biomart,283,"CREATE TABLE ""biomart"" (\n""Gene stable ID"" TEX..."
3,table,DataAttributes,DataAttributes,8357,CREATE TABLE DataAttributes(DAID INT PRIMARY K...


In [6]:
biomart.to_sql("biomart", con, index=False, if_exists="append")

C:\Users\tanub\miniconda3\lib\site-packages\pandas\core\generic.py:2778: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [15]:
df1 = pd.read_sql("""
            select distinct(b.DataName), b.DID, a."Gene name", a."Gene description" 
            from biomart a join Data b on a."Gene stable ID"=b.DataName
""", con)
df1

,DataName,DID,Gene name,Gene description
0,AT3G18710,1985,PUB29,U-box domain-containing protein 29 [Source:Uni...
1,AT4G25880,10489,APUM6,"Pumilio homolog 6, chloroplastic [Source:UniPr..."
2,AT1G71695,10490,PER12,Peroxidase 12 [Source:UniProtKB/Swiss-Prot;Acc...
3,AT5G41480,10491,EMB9,Folylpolyglutamate synthetase family protein [...
4,AT5G15008,10492,None,unknown protein; Has 30201 Blast hits to 17322...
...,...,...,...,...
27411,AT2G20860,27580,LIP1,"Lipoyl synthase, mitochondrial [Source:UniProt..."
27412,AT3G14210,6604,ESM1,GDSL esterase/lipase ESM1 [Source:UniProtKB/Sw...
27413,AT5G01920,3647,STN8,"Serine/threonine-protein kinase STN8, chloropl..."
27414,AT2G26280,5096,CID7,CTC-interacting domain 7 [Source:TAIR;Acc:AT2G...


In [35]:
df2 = pd.read_sql("""
            select distinct(b.DataName), b.DID, a."GO term name"  
            from biomart a join Data b on a."GO term accession"=b.DataName
""", con)
df2

,DataName,DID,GO term name
0,GO:0005488,28662,binding
1,GO:0010200,28663,response to chitin
2,GO:0004842,28664,ubiquitin-protein transferase activity
3,GO:0016874,28665,ligase activity
4,GO:0016567,28666,protein ubiquitination
...,...,...,...
6141,GO:0015148,34803,D-xylose transmembrane transporter activity
6142,GO:0015753,34804,D-xylose transport
6143,GO:0015591,34805,D-ribose transmembrane transporter activity
6144,GO:0080131,34806,hydroxyjasmonate sulfotransferase activity


In [28]:
genename = df1[["DID","Gene name"]]
genename.dropna(subset=['Gene name'], inplace=True)
genename1 = genename.drop_duplicates()
genename1.columns = [["DID_Data", "DataValue"]]
genename1.insert(0, 'DataAttributeType', 'GeneName')
genename1 #all gene names df

<ipython-input-28-dedbf7bedf66>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genename.dropna(subset=['Gene name'], inplace=True)


,DataAttributeType,DID_Data,DataValue
0,GeneName,1985,PUB29
1,GeneName,10489,APUM6
2,GeneName,10490,PER12
3,GeneName,10491,EMB9
7,GeneName,714,KAT1
...,...,...,...
27410,GeneName,27579,LON3
27411,GeneName,27580,LIP1
27412,GeneName,6604,ESM1
27413,GeneName,3647,STN8


In [33]:
genedesc = df1[["DID", "Gene description"]]
genedesc.dropna(subset=['Gene description'], inplace=True)
genedesc1 = genedesc.drop_duplicates()
genedesc1.columns = [["DID_Data", "DataValue"]]
genedesc1.insert(0, 'DataAttributeType', 'GeneDescription')
genedesc1 #all gene desc df

<ipython-input-33-8ce726bea328>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genedesc.dropna(subset=['Gene description'], inplace=True)


,DataAttributeType,DID_Data,DataValue
0,GeneDescription,1985,U-box domain-containing protein 29 [Source:Uni...
1,GeneDescription,10489,"Pumilio homolog 6, chloroplastic [Source:UniPr..."
2,GeneDescription,10490,Peroxidase 12 [Source:UniProtKB/Swiss-Prot;Acc...
3,GeneDescription,10491,Folylpolyglutamate synthetase family protein [...
4,GeneDescription,10492,unknown protein; Has 30201 Blast hits to 17322...
...,...,...,...
27411,GeneDescription,27580,"Lipoyl synthase, mitochondrial [Source:UniProt..."
27412,GeneDescription,6604,GDSL esterase/lipase ESM1 [Source:UniProtKB/Sw...
27413,GeneDescription,3647,"Serine/threonine-protein kinase STN8, chloropl..."
27414,GeneDescription,5096,CTC-interacting domain 7 [Source:TAIR;Acc:AT2G...


In [43]:
gotermname = df2[["DID", "GO term name"]]
gotermname.columns = [["DID_Data", "DataValue"]]
gotermname.insert(0, 'DataAttributeType', 'GOTermName')
gotermname #all go term names df

,DataAttributeType,DID_Data,DataValue
0,GOTermName,28662,binding
1,GOTermName,28663,response to chitin
2,GOTermName,28664,ubiquitin-protein transferase activity
3,GOTermName,28665,ligase activity
4,GOTermName,28666,protein ubiquitination
...,...,...,...
6141,GOTermName,34803,D-xylose transmembrane transporter activity
6142,GOTermName,34804,D-xylose transport
6143,GOTermName,34805,D-ribose transmembrane transporter activity
6144,GOTermName,34806,hydroxyjasmonate sulfotransferase activity


In [47]:
df3 = genename1.append(genedesc1, ignore_index=True)
df3.count() #gene names + gene description

DataAttributeType    39979
DID_Data             39979
DataValue            39979
dtype: int64

In [49]:
df4 = df3.append(gotermname, ignore_index=True)
df4.count() #gene names + gene description + go term names

DataAttributeType    46125
DID_Data             46125
DataValue            46125
dtype: int64

In [50]:
df4.insert(0, 'DAID', range(1, 1+len(df4)))
df4 #df for the DataAtrributes table

,DAID,DataAttributeType,DID_Data,DataValue
0,1,GeneName,1985,PUB29
1,2,GeneName,10489,APUM6
2,3,GeneName,10490,PER12
3,4,GeneName,10491,EMB9
4,5,GeneName,714,KAT1
...,...,...,...,...
46120,46121,GOTermName,34803,D-xylose transmembrane transporter activity
46121,46122,GOTermName,34804,D-xylose transport
46122,46123,GOTermName,34805,D-ribose transmembrane transporter activity
46123,46124,GOTermName,34806,hydroxyjasmonate sulfotransferase activity


In [79]:
#c.execute("""
#        DROP TABLE DataAttributes
#""")

In [95]:
pd.read_sql("""
            SELECT * FROM sqlite_master
            WHERE type='table';
""", con)

,type,name,tbl_name,rootpage,sql
0,table,DataType,DataType,2,CREATE TABLE DataType(DTID INT PRIMARY KEY NOT...
1,table,Data,Data,4,CREATE TABLE Data(DID INT PRIMARY KEY NOT NULL...
2,table,biomart,biomart,283,"CREATE TABLE ""biomart"" (\n""Gene stable ID"" TEX..."
3,table,DataAttributes,DataAttributes,8357,CREATE TABLE DataAttributes(DAID INT PRIMARY K...


In [81]:
c.execute("""
            CREATE TABLE DataAttributes(DAID INT PRIMARY KEY NOT NULL,
            DataAttributeType TEXT, DID_Data INT, DataValue TEXT,
            FOREIGN KEY(DID_Data) REFERENCES Data(DID))
""")

In [84]:
df4.columns = ['DAID', 'DataAttributeType', 'DID_Data', 'DataValue']
df4

,DAID,DataAttributeType,DID_Data,DataValue
0,1,GeneName,1985,PUB29
1,2,GeneName,10489,APUM6
2,3,GeneName,10490,PER12
3,4,GeneName,10491,EMB9
4,5,GeneName,714,KAT1
...,...,...,...,...
46120,46121,GOTermName,34803,D-xylose transmembrane transporter activity
46121,46122,GOTermName,34804,D-xylose transport
46122,46123,GOTermName,34805,D-ribose transmembrane transporter activity
46123,46124,GOTermName,34806,hydroxyjasmonate sulfotransferase activity


I was getting an error while copying the dataframe df4 to my database table DataAttributes, it was saying that the columns does not exist in table and the column names showing was in quotes, so I decided to first create a csv file of the dataframe and then read that csv file into the DataAttributes Table of database and it worked fine.

In [85]:
df4.to_sql("DataAttributes", con, index=False, if_exists="append")

In [94]:
pd.read_sql("""
            select * from DataAttributes
""", con)

,DAID,DataAttributeType,DID_Data,DataValue
0,1,GeneName,1985,PUB29
1,2,GeneName,10489,APUM6
2,3,GeneName,10490,PER12
3,4,GeneName,10491,EMB9
4,5,GeneName,714,KAT1
...,...,...,...,...
46120,46121,GOTermName,34803,D-xylose transmembrane transporter activity
46121,46122,GOTermName,34804,D-xylose transport
46122,46123,GOTermName,34805,D-ribose transmembrane transporter activity
46123,46124,GOTermName,34806,hydroxyjasmonate sulfotransferase activity


In [96]:
con.close()